In [27]:
%pip install langchain_upstage langchain_community langchain_pinecone pinecone docx2txt langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchainhub]
Note: you may need to restart the kernel to use updated packages.


In [ ]:

# embedding
from langchain_upstage import UpstageEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
upsatge_api_key = os.getenv("UPSTAGE_API_KEY")

embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = 'table-markdown-index'

# Split documents into smaller chunks
loader = Docx2txtLoader("./tax_with_markdown.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
document_list = loader.load_and_split(text_splitter=text_splitter)
chunked_documents = text_splitter.split_documents(document_list)

# Initialize the PineconeVectorStore
database = PineconeVectorStore.from_documents(
    documents=[],  # Start with an empty list
    embedding=embedding,
    index_name=index_name
)

# Upload documents in batches
batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size: {batch_size}')
    batch = chunked_documents[i:i + batch_size]
    database.add_documents(batch)  # Add documents to the existing database

query = "연봉 9천만원 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

from langchain_upstage import ChatUpstage

llm = ChatUpstage()

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

retriever = database.as_retriever(search_kwargs={"k": 4})

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_messages(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 마세요.
    그런 경우에는 질문만 리턴해주세요.
                                          
    사전: {dictionary}
    사용자의 질문: {query}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain



index: 0, batch size: 100
index: 100, batch size: 100
index: 200, batch size: 100
index: 300, batch size: 100


In [9]:
len(chunked_documents)

336

In [10]:
query = "연봉 9천만원 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

In [11]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [12]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/jhlee/.pyenv/versions/3.10.17/envs/streamlit/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
retriever = database.as_retriever(search_kwargs={"k": 4})

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [15]:
ai_message = qa_chain.invoke({"query": query})

In [16]:
ai_message

{'query': '연봉 9천만원 직장인의 소득세는 얼마인가요?',
 'result': '연봉 9천만원 직장인의 소득세는 1,574만원입니다. 이는 5,000만원 초과 8,800만원 이하 구간에 해당하며, 세율은 24%입니다. 따라서 (5,000만원을 초과하는 금액인) 4,000만원의 24%에 해당하는 금액과 624만원을 더한 금액이 소득세가 됩니다.'}

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_messages(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 마세요.
    그런 경우에는 질문만 리턴해주세요.
                                          
    사전: {dictionary}
    사용자의 질문: {query}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain


In [25]:
new_question = dictionary_chain.invoke({"question": query})

In [26]:
new_question

'사용자의 질문은 "연봉 9천만원 직장인의 소득세는 얼마인가요?"입니다. 이 질문에 대한 정확한 답변은 사용자의 다양한 개인 상황에 따라 달라질 수 있습니다. 소득세는 과세표준에 따라 계산되며, 과세표준은 연봉에서 각종 공제를 제외한 금액입니다. 따라서, 정확한 소득세를 알기 위해서는 연봉에서 각종 공제 항목(예: 근로소득공제, 인적공제, 연금보험료공제 등)을 차감한 후, 해당 과세표준에 맞는 세율을 적용해야 합니다.\n\n일반적으로 연봉 9천만원 정도의 직장인의 경우, 대략적인 소득세를 계산해보면 다음과 같습니다:\n\n1. 과세표준 계산: 연봉 9천만원에서 각종 공제를 차감한 금액\n2. 세율 적용: 과세표준에 따른 세율을 적용\n3. 세액 계산: 적용된 세율에 따라 소득세액을 계산\n\n그러나 이는 대략적인 계산이며, 실제 소득세는 사용자의 개인 상황에 따라 다를 수 있습니다. 정확한 소득세를 알고 싶다면, 국세청 홈페이지의 소득세 계산기를 사용하거나 세무사에게 상담을 받는 것이 좋습니다.'